In [7]:
import pandas as pd
import datetime as dt
import xm_info as xi

a = xi.manualQuery()
print('\nRESULTADO: \n')
a

In [5]:
import pandas as pd
import datetime as dt
import xm_query as xq

item = 'Recursos'
sd = dt.date(2020,1,29)
ed = dt.date(2020,1,30)
freq = 'Horaria'
var = ['Generacion Ideal','Generacion Real']

a = xq.xmQueryAPI(item, sd, ed, freq, var=[], save=False)
print('\nRESULTADO: \n')
a